In [1]:
import os
import sys
os.chdir('/local/home/dhaziza/entrack')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"
sys.path.append('/local/home/dhaziza/entrack/')

import src.features as ft_def
import csv
import re
import glob
import pickle
import time
import xml.etree.ElementTree as ET

class NoSuchXMLElementException(Exception):
    pass

def intvals(s):
    vals = re.findall(r'\d+', s)
    return [int(i) for i in vals]

def intval(s):
    vals = intvals(s)
    return vals[0]

def boolval(cond):
    return 1 if cond else 0

def print_stats(dataset, name, field, cond):
    subject_ids = set()
    num = 0.0
    total_value = 0.0
    for row in dataset:
        if cond(row):
            num += 1
            total_value += row[field]
            subject_ids.add(row[ft_def.STUDY_PATIENT_ID])

    if num == 0:
        print('!! NO DATA FOR %s' % name)
        return
    value_mean = total_value/num

    value_std = 0.0
    for row in dataset:
        if cond(row):
            value_std += (row[field] - value_mean)*(row[field] - value_mean)
    value_std /= num
    print('%s Mean %s %f, variance %f [%d images / %d unique patients]' % (
        name, field, value_mean, value_std, num, len(subject_ids)))

In [2]:
# Koln data
csv_orig = 'data/raw/csv/orig/koln.csv'
csv_output = 'data/raw/csv/koln.csv'

data = []
with open(csv_orig) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({
            ft_def.AGE: int(row['Alter']),
            'health_pd': 1,
            ft_def.HEALTHY: 0,
            ft_def.SEX: int(row['Geschlecht (male =1; female =2)'])-1,
            ft_def.STUDY_PATIENT_ID: intval(row['ID'])
        })

with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print_stats(data, 'KOLN', 'age', lambda r: True)

KOLN Mean age 63.546875, variance 83.904053 [128 images / 128 unique patients]


In [3]:
# Load ADNI/AIBL features from XML
def load_image_features_from_xml(path, image_id_prefix=''):
    image_id_to_feature = {}
    all_files = glob.glob(path)
    print('Loading %d XML files... Please be patient! :)' % len(all_files))
    begin_time = time.time()
    milestone = 100
    for i, f in enumerate(all_files):
        if i >= milestone:
            milestone *= 2
            print('  .. done %d in %ds' % (i, time.time() - begin_time))
        tree = ET.parse(f)
        root = tree.getroot()
        def elem_unique(path):
            e = root.findall(path)
            if len(e) == 0:
                raise NoSuchXMLElementException()
            assert(len(e) == 1)
            return e[0]
        image_id_text = elem_unique("./project/subject/study/imagingProtocol/imageUID").text
        sex_text = elem_unique("./project/subject/subjectSex").text
        age_text = elem_unique("./project/subject/study/subjectAge").text
        research_group = elem_unique("./project/subject/researchGroup").text
        image_id = int(elem_unique("./project/subject/study/imagingProtocol/imageUID").text)
        subject_id_text = elem_unique("./project/subject/subjectIdentifier").text
        manufacturer = elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Manufacturer']").text
        field_strength = elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Field Strength']").text
        try:
            weighting = elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Weighting']").text
        except NoSuchXMLElementException:
            weighting = 'DTI'
        image_id_to_feature['%s%s' % (image_id_prefix, image_id)] = {
            ft_def.AGE: intval(age_text),
            ft_def.SEX: 0 if sex_text == 'M' else 1,
            ft_def.STUDY_IMAGE_ID: int(image_id_text),
            ft_def.SUBJECT_LABEL: subject_id_text,
            ft_def.MRI_MANUFACTURER: manufacturer,
            ft_def.MRI_FIELD_STRENGTH: field_strength,
        }
    return image_id_to_feature

adni_aibl_features_from_xml = load_image_features_from_xml('/local/ADNI_AIBL/metadata/ADNI/*.xml')
adni_aibl_features_from_xml.update(load_image_features_from_xml(
    '/local/ADNI_AIBL/metadata/AIBL/*.xml', image_id_prefix='A'))

Loading 87606 XML files... Please be patient! :)
  .. done 100 in 0s
  .. done 200 in 0s
  .. done 400 in 0s
  .. done 800 in 1s
  .. done 1600 in 1s
  .. done 3200 in 3s
  .. done 6400 in 6s
  .. done 12800 in 13s
  .. done 25600 in 26s
  .. done 51200 in 50s
Loading 3419 XML files... Please be patient! :)
  .. done 100 in 0s
  .. done 200 in 0s
  .. done 400 in 0s
  .. done 800 in 0s
  .. done 1600 in 1s
  .. done 3200 in 2s


In [4]:
# ADNI AIBL
csv_orig = 'data/raw/csv/orig/loni_adni_aibl.csv'
adni_aibl_labels = pickle.load(open('/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_class_labels_T1.pkl', 'r'))
adni_aibl_valid_set = pickle.load(open('/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_valid_T1_NC_AD.pkl', 'r'))

ADNI_AIBL_GROUPS = ['Normal', 'MCI', 'AD', 'EMCI', 'LMCI', 'SMC']
def convert_adni_aibl(csv_output, cond):
    print('ADNI_AIBL: Converting to %s' % (csv_output))
    data = []
    with open(csv_orig) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['Project'] not in ['ADNI', 'AIBL']:
                continue
            image_id_sai = image_id = str(row['Image ID'])
            image_id = int(image_id)
            study_id = 0
            if row['Project'] == 'AIBL':
                image_id_sai = 'A' + image_id_sai
                study_id = 1
            if image_id_sai not in adni_aibl_labels:
                continue
            label_id = adni_aibl_labels[image_id_sai]
            group = ADNI_AIBL_GROUPS[label_id]
            if group not in ADNI_AIBL_GROUPS:
                continue
            cur_data = {
                ft_def.SUBJECT_LABEL: row['Subject ID'],
                ft_def.IMAGE_LABEL: image_id_sai,
                ft_def.AGE: intval(row['Age']),
                'health_ad': boolval(group == 'AD'),
                'health_emci': boolval(group == 'EMCI'),
                'health_lmci': boolval(group == 'LMCI'),
                'health_mci': boolval(group == 'MCI'),
                'health_smc': boolval(group == 'SMC'),
                ft_def.HEALTHY: boolval(group == 'Normal'),
                ft_def.SEX: 0 if row['Sex'] == 'M' else 1,
                ft_def.STUDY_IMAGE_ID: image_id,
                ft_def.STUDY_PATIENT_ID: hash(row['Subject ID']) % 100000000,
                ft_def.STUDY_ID: study_id,
                ft_def.DATASET: 'test' if image_id_sai in adni_aibl_valid_set else 'train',
            }
            cur_data.update(adni_aibl_features_from_xml[image_id_sai])
            if not cond(row, cur_data):
                continue
            data.append(cur_data)

    with open(csv_output, "wb+") as csvfile:
        writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
        writer.writeheader()
        for row in data:
            writer.writerow(row)

    name = os.path.basename(csv_output).split('.csv')[0]
    print_stats(data, '*', 'age', lambda x: True)
    for ft in ['health_ad', ft_def.HEALTHY]:
        print_stats(data, ft, 'age', lambda row: row[ft] == 1)
        print_stats(data, 'SAI_TEST/%s' % ft, 'age', lambda row: row[ft] == 1 and row['dataset'] == 'test')
        print_stats(data, 'SAI_TRAIN/%s' % ft, 'age', lambda row: row[ft] == 1 and row['dataset'] == 'train')
        print_stats(data, 'ADNI/%s' % ft, 'age', lambda row: row[ft] == 1 and row['study_id'] == 0)
        print_stats(data, 'AIBL/%s' % ft, 'age', lambda row: row[ft] == 1 and row['study_id'] == 1)
    print('')

def is_ad_or_hc(f):
    return f['health_ad'] == 1 or f[ft_def.HEALTHY] == 1

convert_adni_aibl('data/raw/csv/adni_aibl.csv', lambda r, f: True)
convert_adni_aibl('data/raw/csv/adni_aibl__ad_hc.csv', lambda r, f: is_ad_or_hc(f))

ADNI_AIBL: Converting to data/raw/csv/adni_aibl.csv
* Mean age 74.851736, variance 53.252787 [19762 images / 2449 unique patients]
health_ad Mean age 75.643432, variance 58.140956 [4476 images / 742 unique patients]
SAI_TEST/health_ad Mean age 76.192500, variance 51.725444 [400 images / 70 unique patients]
SAI_TRAIN/health_ad Mean age 75.589549, variance 58.738056 [4076 images / 672 unique patients]
ADNI/health_ad Mean age 75.687428, variance 57.892841 [4335 images / 646 unique patients]
AIBL/health_ad Mean age 74.290780, variance 63.879986 [141 images / 96 unique patients]
healthy Mean age 75.355394, variance 39.633441 [6618 images / 1048 unique patients]
SAI_TEST/healthy Mean age 75.210000, variance 39.330900 [400 images / 62 unique patients]
SAI_TRAIN/healthy Mean age 75.364748, variance 39.651456 [6218 images / 986 unique patients]
ADNI/healthy Mean age 75.694186, variance 38.329251 [5814 images / 591 unique patients]
AIBL/healthy Mean age 72.905473, variance 42.232358 [804 images 

In [5]:
# Erasmus data
csv_orig = '/local/ERSM/ADNI_Diagnosis_mciconv.csv'

ERASMUS_ADNI_GROUPS = ['CN', 'AD', 'MCI']
def convert_erasmus_adni(csv_output, cond):
    print('ERASMUS_ADNI: Converting to %s' % (csv_output))
    data = []
    i = 0
    with open(csv_orig) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Reconstruct a single int subject id from something like '002_S_0295'
            subject_label = row['PTID']
            group = row['Diagnosis']
            if group not in ERASMUS_ADNI_GROUPS:
                continue
            cur_data = {
                'health_ad': boolval(group == 'AD'),
                'health_mci': boolval(group == 'MCI'),
                ft_def.HEALTHY: boolval(group == 'CN'),
                ft_def.IMAGE_LABEL: subject_label,
                ft_def.SUBJECT_LABEL: subject_label,
                ft_def.STUDY_PATIENT_ID: i,
            }
            i += 1
            if not cond(row, cur_data):
                continue
            data.append(cur_data)

    with open(csv_output, "wb+") as csvfile:
        writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
        writer.writeheader()
        for row in data:
            writer.writerow(row)

    name = os.path.basename(csv_output).split('.csv')[0]
    print_stats(data, '*', 'healthy', lambda x: True)
    for ft in ['health_ad', 'health_mci', ft_def.HEALTHY]:
        print_stats(data, ft, ft, lambda row: row[ft] == 1)
    print('')

convert_erasmus_adni('data/raw/csv/erasmus_adni.csv', lambda r, d: True)

ERASMUS_ADNI: Converting to data/raw/csv/erasmus_adni.csv
* Mean healthy 0.240069, variance 0.182436 [1737 images / 1737 unique patients]
health_ad Mean health_ad 1.000000, variance 0.000000 [342 images / 342 unique patients]
health_mci Mean health_mci 1.000000, variance 0.000000 [978 images / 978 unique patients]
healthy Mean healthy 1.000000, variance 0.000000 [417 images / 417 unique patients]



In [6]:
# ADNI/AIBL: sai dataset
train_ad_nc = pickle.load(open('/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_train_T1_NC_AD.pkl', 'r'))
valid_ad_nc = pickle.load(open('/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_valid_T1_NC_AD.pkl', 'r'))
csv_output = 'data/raw/csv/adni_aibl__ad_hc__sai.csv'

print('ADNI_AIBL[sai_ad_nc]: Converting to %s' % (csv_output))
data = []
for image_id_sai, is_ad in train_ad_nc.items():
    cur_data = adni_aibl_features_from_xml[image_id_sai.split('_')[0]]
    cur_data.update({
        ft_def.IMAGE_LABEL: image_id_sai,
        'health_ad': is_ad,
        ft_def.HEALTHY: boolval(is_ad == 0),
        ft_def.DATASET: 'train',
    })
    data.append(cur_data)
for image_id_sai, is_ad in valid_ad_nc.items():
    cur_data = adni_aibl_features_from_xml[image_id_sai.split('_')[0]]
    cur_data.update({
        ft_def.IMAGE_LABEL: image_id_sai,
        'health_ad': is_ad,
        ft_def.HEALTHY: boolval(is_ad == 0),
        ft_def.DATASET: 'test',
    })
    data.append(cur_data)

with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)

ADNI_AIBL[sai_ad_nc]: Converting to data/raw/csv/adni_aibl__ad_hc__sai.csv


In [7]:
# PPMI - datakey is image_id
# Example:
# <subject_id>/Axial_PD-T2_TSE_FS/2013-04-09_09_24_46.0/S<serie_id>/
# ... PPMI_4139_MR_Axial_PD-T2_TSE_FS_br_raw_20130625124532171_76_S<serie_id>_I<image_id>.nii

csv_output = 'data/raw/csv/ppmi.csv'

image_sizes = {}
ppmi_path = '/local/PPMI/raw/'
data = []
for f in glob.glob(ppmi_path + '*.xml'):
    tree = ET.parse(f)
    root = tree.getroot()
    def elem_unique(path):
        e = root.findall(path)
        if len(e) == 0:
            raise NoSuchXMLElementException()
        assert(len(e) == 1)
        return e[0]
    sex_text = elem_unique("./project/subject/subjectSex").text
    age_text = elem_unique("./project/subject/study/subjectAge").text
    research_group = elem_unique("./project/subject/researchGroup").text
    image_id_text = elem_unique("./project/subject/study/imagingProtocol/imageUID").text
    subject_id_text = elem_unique("./project/subject/subjectIdentifier").text
    post_mortem_text = elem_unique("./project/subject/study/postMortem").text
    try:
        weighting = elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Weighting']").text
    except NoSuchXMLElementException:
        weighting = 'DTI'
    assert(post_mortem_text == 'F')
    assert(weighting in ['T1', 'T2', 'DTI'])
    if research_group not in ['Control', 'PD', 'GenCohort PD', 'GenCohort Unaff', 'Prodromal', 'SWEDD']:
        continue
    data.append({
        ft_def.AGE: intval(age_text),
        'health_pd': boolval(research_group == 'PD'),
        'health_prodromal': boolval(research_group == 'Prodromal'),
        'health_swedd': boolval(research_group == 'SWEDD'),
        'health_gencohort_unaff': boolval(research_group == 'GenCohort Unaff'),
        'health_gencohort_pd': boolval(research_group == 'GenCohort PD'),
        ft_def.HEALTHY: boolval(research_group == 'Control'),
        ft_def.SEX: 0 if sex_text == 'M' else 1,
        ft_def.STUDY_IMAGE_ID: int(image_id_text),
        ft_def.STUDY_PATIENT_ID: int(subject_id_text),
        ft_def.SUBJECT_LABEL: subject_id_text,
    })
    # Also count how many samples per image size
    img_size = (
        elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Matrix X']").text,
        elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Matrix Y']").text,
        elem_unique("./project/subject/study/imagingProtocol/protocolTerm/protocol[@term='Matrix Z']").text,
    )
    if not img_size in image_sizes:
        image_sizes[img_size] = 0
    image_sizes[img_size] += 1

with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)

print('%s different sizes found!' % (len(image_sizes)))
print_stats(data, 'PPMI', 'age', lambda x: True)
print_stats(data, 'PPMI/PD', 'age', lambda x: x['health_pd'])
print_stats(data, 'PPMI/Control', 'age', lambda x: x[ft_def.HEALTHY])

194 different sizes found!
PPMI Mean age 61.572225, variance 96.216637 [3766 images / 990 unique patients]
PPMI/PD Mean age 61.389093, variance 94.107652 [1907 images / 402 unique patients]
PPMI/Control Mean age 60.088195, variance 126.102127 [737 images / 183 unique patients]
